# UFC Prediction -- Scraping Notebook

This notebook includes all the code used to scrape the data from various websites for the UFC Prediction project. 

Because I had to connect events over multiple websites, I started in excel. The beginning of this workbook starts with this csv. This csv included event titles, ufcsts.com event urls, and bestfightodds.com event urls for the corresponding events. 

# Scraping BestFightOdds.com

In [1]:
# Load Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil      
import datetime
from scipy.stats import norm
import xgboost
from xgboost import XGBClassifier
from random import randint
import  random
import os
os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import warnings
warnings.filterwarnings('ignore')
from random import randint
from cmath import nan
import urllib
import urllib.request
import re
import time

c:\Users\Travis\anaconda3\envs\Trav310\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Load Excel-fixed CSV

This contains the URLS necessary to scrape from multiple websites.

In [2]:
fight_odds_urls = pd.read_csv('data/final/events/Final_Hand_Done_BFO_Urls_with UFC_Stats_Urls.csv')

print(fight_odds_urls.shape)
fight_odds_urls.head(3)

(508, 7)


,event_title,event_url,event_odds_url,Short_title,Short_Title_2 (vs. to vs),Name1,Name2
0,UFC Fight Night: Thompson vs. Holland,http://ufcstats.com/event-details/b23388ff8ac6...,https://www.bestfightodds.com/events/ufc-fight...,Thompson vs. Holland,Thompson vs Holland,thompson,holland
1,UFC Fight Night: Nzechukwu vs. Cutelaba,http://ufcstats.com/event-details/012fc7cd0779...,https://www.bestfightodds.com/events/ufc-fight...,Nzechukwu vs. Cutelaba,Nzechukwu vs Cutelaba,nzechukwu,cutelaba
2,UFC 281: Adesanya vs. Pereira,http://ufcstats.com/event-details/b3b6e80b7d5f...,https://www.bestfightodds.com/events/ufc-281-2529,Adesanya vs. Pereira,Adesanya vs Pereira,adesanya,pereira


In [3]:
# get all event urls from the fight_odds_urls dataframe
event_urls = fight_odds_urls['event_url'].unique()
print(f' {len(event_urls)} events')

 508 events


#### Scraping BFO via Requests & Headers

You can only scrape bestfightodds if you add in a browser heading.

For example:

In [4]:
url = 'https://www.bestfightodds.com/events/ufc-282-2621'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
r = requests.get(url, headers = header)
dfs = pd.read_html(r.text)
dfs[0]

,Jan Brachowicz
0,Magomed Ankalaev
1,Over 1½ rounds
2,Under 1½ rounds
3,Over 2½ rounds
4,Under 2½ rounds
...,...
1725,Under 7½ fights go the distance
1726,Over 8½ fights go the distance
1727,Under 8½ fights go the distance
1728,Over 9½ fights go the distance


## Scraping BFO Data

In [5]:
all_event_odds_urls = fight_odds_urls.event_odds_url.unique()
len(all_event_odds_urls)

489

In [6]:
len('https://www.bestfightodds.com/events/')

37

In [7]:
def get_odds(url):
        # you can scrape by pretending to be a browser
        header = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest"
        }
        r = requests.get(url, headers = header)
        dfs = pd.read_html(r.text)
        data = dfs[1]
        data.rename(columns={'Unnamed: 0': 'fighter'}, inplace=True)
        data['event_odds_url'] = url
        event_name = url.split('/')[4]
        data['event_id'] = event_name
        data.to_csv('data/final/events/odds_per_event/' + event_name + '.csv', index=False)
        return data



In [8]:
get_odds(all_event_odds_urls[0]).head(2)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,Bet365,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id
0,Kevin Holland,-170▲,-185▼,-195▼,-200▼,-166▼,NaN,-200▼,-175▲,-188▲,NaN,-200▼,NaN,118.0,NaN,https://www.bestfightodds.com/events/ufc-fight...,ufc-fight-night-214-2607
1,Stephen Thompson,+145▼,+150▲,+165▲,+160▲,+140▲,NaN,+160▲,+150▼,+150▼,NaN,+170▲,NaN,118.0,NaN,https://www.bestfightodds.com/events/ufc-fight...,ufc-fight-night-214-2607


In [9]:
# drop nan
all_event_odds_urls = all_event_odds_urls[~pd.isnull(all_event_odds_urls)]
all_event_odds_urls

array(['https://www.bestfightodds.com/events/ufc-fight-night-214-2607',
       'https://www.bestfightodds.com/events/ufc-fight-night-215-2633',
       'https://www.bestfightodds.com/events/ufc-281-2529',
       'https://www.bestfightodds.com/events/ufc-fight-night-214ting-championship-2612',
       'https://www.bestfightodds.com/events/ufc-fight-night-213-2606',
       'https://www.bestfightodds.com/events/ufc-281-2586',
       'https://www.bestfightodds.com/events/ufc-fight-night-212-2579',
       'https://www.bestfightodds.com/events/ufc-vegas-57-2489',
       'https://www.bestfightodds.com/events/ufc-fight-night-211-2569',
       'https://www.bestfightodds.com/events/ufc-279-2541',
       'https://www.bestfightodds.com/events/ufc-fight-night-gane-vs-tuivasa-2518',
       'https://www.bestfightodds.com/events/ufc-278-usman-vs-edwards-2-2545',
       'https://www.bestfightodds.com/events/ufc-fight-night-vera-vs-cruz-2552',
       'https://www.bestfightodds.com/events/ufc-on-espn-santo

#### Now, I check to see if the event has already been scraped. If it has, I skip it. If not, I scrape it.

In [10]:
n=0
errors = []
complete = []
files = os.listdir('data/final/events/odds_per_event/')
file_list = [n[:-4] for n in files]

for url in all_event_odds_urls:
    url_file_name = url.split('/')[-1]
    url_file_name2 = url_file_name[:-4]
    if url_file_name not in file_list:
        try:
            get_odds(url)
            print(f'{n} / {len(all_event_odds_urls)}')
            n+=1
        except:
            print(f'ERROR! {n} / {len(all_event_odds_urls)}')
            errors.append(url)
            n+=1
    else:
        print(f'{url_file_name} already exists')
        complete.append(url_file_name)

print(f' total errors: {len(errors)}')

ufc-fight-night-214-2607 already exists
ufc-fight-night-215-2633 already exists
ufc-281-2529 already exists
ufc-fight-night-214ting-championship-2612 already exists
ufc-fight-night-213-2606 already exists
ufc-281-2586 already exists
ufc-fight-night-212-2579 already exists
ufc-vegas-57-2489 already exists
ufc-fight-night-211-2569 already exists
ufc-279-2541 already exists
ufc-fight-night-gane-vs-tuivasa-2518 already exists
ufc-278-usman-vs-edwards-2-2545 already exists
ufc-fight-night-vera-vs-cruz-2552 already exists
ufc-on-espn-santos-vs-hill-2534 already exists
ufc-277-pena-vs-nunes-2-2517 already exists
ufc-fight-night-blaydes-vs-aspinall-2519 already exists
ufc-on-abc-ortega-vs-rodriguez-2524 already exists
ufc-on-espn-dos-anjos-vs-fiziev-2501 already exists
ufc-276-adesanya-vs-cannonier-2478 already exists
ufc-on-espn-tsarukyan-vs-gamrot-2574 already exists
ufc-fight-night-kattar-vs-emmett-2523 already exists
ufc-275-teixeira-vs-prochazka-2561 already exists
ufc-fight-night-211-244

### Append Together

In [11]:
#append 
files = os.listdir('data/final/events/odds_per_event/')

In [12]:
all_event_odds = []

for file in files:
    df = pd.read_csv('data/final/events/odds_per_event/' + file)
    # append
    all_event_odds.append(df)

all_event_odds = pd.concat(all_event_odds)

print(all_event_odds.shape)
all_event_odds.head()

(331831, 17)


,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+205▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
3,Thiago Alves,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+250▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
4,Dan Henderson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-200▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN


### Adding Event Names & URLs to Event_Odds df

In [13]:
fight_odds_urls.head(3)

,event_title,event_url,event_odds_url,Short_title,Short_Title_2 (vs. to vs),Name1,Name2
0,UFC Fight Night: Thompson vs. Holland,http://ufcstats.com/event-details/b23388ff8ac6...,https://www.bestfightodds.com/events/ufc-fight...,Thompson vs. Holland,Thompson vs Holland,thompson,holland
1,UFC Fight Night: Nzechukwu vs. Cutelaba,http://ufcstats.com/event-details/012fc7cd0779...,https://www.bestfightodds.com/events/ufc-fight...,Nzechukwu vs. Cutelaba,Nzechukwu vs Cutelaba,nzechukwu,cutelaba
2,UFC 281: Adesanya vs. Pereira,http://ufcstats.com/event-details/b3b6e80b7d5f...,https://www.bestfightodds.com/events/ufc-281-2529,Adesanya vs. Pereira,Adesanya vs Pereira,adesanya,pereira


In [14]:
def get_event_title(event_odds_url):
    # even[event_name] is the event name
    # even[event_odds_url] is the url
    try:
        data = fight_odds_urls[fight_odds_urls['event_odds_url'] == event_odds_url]
        event_name = data.event_title.values[0]
        return event_name
    except:
        return nan

    

In [15]:
def get_ufcstats_url(event_odds_url):
    try:
        data = fight_odds_urls[fight_odds_urls['event_odds_url'] == event_odds_url]
        event_url = data.event_url.values[0]
        return event_url
    except:
        return nan

#### Dropping Unnecessary Data

I do not need the specific odds for everything in a fight, just the odds to win.

In [16]:
# drop rows if  'decision' in the fighter column
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('decision', na=False)]
# drop rows if 'result' in the fighter column
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('result', na=False)]
# drop if 'distance' in the fighter column
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('distance', na=False)]

print(all_event_odds.shape)
all_event_odds.head()

(177999, 17)


,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+205▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
3,Thiago Alves,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+250▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN
4,Dan Henderson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-200▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN


In [17]:
all_event_odds['event_name'] = all_event_odds.apply(lambda row: get_event_title(row['event_odds_url']), axis=1)

In [18]:
# check nan
all_event_odds[all_event_odds['event_name'].isnull()]

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365,event_name
0,Jan Brachowicz,+260▲,+260▼,+280▲,+285▲,+270▼,NaN,+285▲,+275▲,+285▼,+275▼,NaN,117.0,NaN,https://www.bestfightodds.com/events/ufc-282-2621,ufc-282-2621,+275▲,NaN
1,Magomed Ankalaev,-315▼,-350▲,-360▼,-375▼,-335▲,NaN,-375▼,-350▼,-345▼,-335▲,NaN,117.0,NaN,https://www.bestfightodds.com/events/ufc-282-2621,ufc-282-2621,-351▼,NaN
2,Over 1½ rounds,NaN,NaN,-270,-286,NaN,NaN,-286,-275,-195,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-282-2621,ufc-282-2621,NaN,NaN
3,Under 1½ rounds,NaN,NaN,+200,+205,NaN,NaN,+205,+200,+168,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-282-2621,ufc-282-2621,NaN,NaN
4,Over 2½ rounds,NaN,NaN,-150▼,-148,-138▼,NaN,-148,-138▼,-130,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-282-2621,ufc-282-2621,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Fight is not a draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-13500,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-fight...,ufc-fight-night-lewis-vs-spivak-2747,NaN,NaN
172,Jesus Santos Aguilar,NaN,+625▼,+700,+650,+700▲,NaN,+650▲,+450,+700▼,+800▼,NaN,6.0,NaN,https://www.bestfightodds.com/events/ufc-fight...,ufc-fight-night-lewis-vs-spivak-2747,NaN,NaN
173,Tatsuro Taira,NaN,-1000▲,-1100,-1000,-1100▼,NaN,-1000▼,-800,-1100▲,-1100▲,NaN,6.0,NaN,https://www.bestfightodds.com/events/ufc-fight...,ufc-fight-night-lewis-vs-spivak-2747,NaN,NaN
176,Fight is a draw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+6500,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-fight...,ufc-fight-night-lewis-vs-spivak-2747,NaN,NaN


In [19]:
all_event_odds['event_ufcstats_url'] = all_event_odds.apply(lambda row: get_ufcstats_url(row['event_odds_url']), axis=1)

In [20]:
all_event_odds.head(3)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365,event_name,event_ufcstats_url
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+205▲,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275▼,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...


In [21]:
all_event_odds.to_csv('data/final/odds/All_Odds_by_Fighter_V1.csv', index=False)

In [22]:
all_event_odds['5D'] = all_event_odds['5D'].str[:-1]
all_event_odds['5D'] = all_event_odds['5D'].astype(float)
all_event_odds.head(3)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365,event_name,event_ufcstats_url
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...


In [23]:
all_event_odds['Ref'] = all_event_odds['Ref'].str[:-1]
all_event_odds['Ref'] = all_event_odds['Ref'].astype(float)
all_event_odds.head(3)

,fighter,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,BetWay,5D,Ref,Props,Props.1,Props.2,event_odds_url,event_id,Bet365,event_name,event_ufcstats_url
0,Brock Lesnar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-225.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
1,Frank Mir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...
2,Georges St-Pierre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-275.0,NaN,NaN,NaN,NaN,https://www.bestfightodds.com/events/ufc-100-137,ufc-100-137,NaN,UFC 100,http://ufcstats.com/event-details/0ee783aa00e4...


In [24]:
cols = ['fighter', '5D', 'Ref', 'event_odds_url', 'event_ufcstats_url', 'event_id', 'event_name']
all_event_odds = all_event_odds[cols]
all_event_odds.head()

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100


In [25]:
# get shape
all_event_odds.shape

(177999, 7)

In [26]:
all_event_odds.to_csv('data/final/odds/All_Odds_by_Fighter_V2.csv', index=False)


## Summary
- ALL ODDS by fighter are available at:
  - data/ufc_BestFightOdds/All_Event_Odds_by_Fighter.csv
- You can combine this with UfcStatsData easily using 'event_ufcstats_url' 

# Add All Events Df

Saved in final/events

In [27]:
all_event_odds = pd.read_csv('data/final/odds/All_Odds_by_Fighter_V2.csv')

In [28]:
# events are located in data/ufc_stats/events2

folder = 'data/ufc_stats/events2/'

# aggregate all events into one dataframe
all_events = []

for file in os.listdir(folder):
    df = pd.read_csv(folder + file)
    # append
    all_events.append(df)

all_events = pd.concat(all_events)

print(all_events.shape)
all_events.head()

(5678, 19)


,Unnamed: 0,W/L,Weight class,Method,Round,Time,Fighter1,Fighter2,F1_Kd,F2_Kd,F1_Str,F2_Str,F1_Td,F2_Td,F1_Sub,F2_Sub,fight_num,event_id,fight_link
0,0,win,Heavyweight,KO/TKO Punches,3,4:23,Ciryl Gane,Tai Tuivasa,1,1,110,29,0,0,0,0,1,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/810c9da0...
1,1,win,Middleweight,U-DEC,3,5:00,Robert Whittaker,Marvin Vettori,0,0,74,33,1,0,0,0,2,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/b8ca1acd...
2,2,win,Middleweight,U-DEC,3,5:00,Nassourdine Imavov,Joaquin Buckley,0,0,51,46,2,0,2,0,3,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/20f316f9...
3,3,win,Middleweight,KO/TKO Punches,3,1:09,Roman Kopylov,Alessio Di Chirico,1,0,59,39,0,0,0,0,4,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/1a191251...
4,4,win,Featherweight,U-DEC,3,5:00,William Gomis,Jarno Errens,0,0,32,20,3,0,0,1,5,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/398db400...


In [29]:
all_events.to_csv('data/final/events/All_Events_V1.csv', index=False)

In [30]:
all_events = pd.read_csv('data/final/events/All_Events_V1.csv')

# Add Odd Changes to Odds

In [31]:
def get_odds(url):
        # you can scrape by pretending to be a browser
    header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
    }
    r = requests.get(url, headers = header)
    dfs = pd.read_html(r.text)
    data = dfs[1]
    data.rename(columns={'Unnamed: 0': 'fighter'}, inplace=True)
    data['event_odds_url'] = url
    event_name = url.split('/')[4]
    data['event_id'] = event_name
    #data.to_csv('data/ufc_BestFightOdds/odds_by_event/' + event_name + '.csv', index=False)

    return data

In [32]:
def get_bfodds(event_url):
    # scrape the changes in odds by event url
    
    driver = webdriver.Chrome()
    # minimize
    driver.minimize_window()
    driver.get(event_url)

    # click show all
    driver.find_element_by_class_name('event-swing-expand').click()
    #wait 2 seconds
    time.sleep(2)

    event_swing_container = driver.find_element_by_id('event-swing-container')


    # the first half of the text is the odds changes, the seconds half are the names. Lets match them up. 
    # first split the text by new line
    event_swing_container.text.split('\n')

    # get the names. You can identify them because they will be more than 5 characters long
    names = [x for x in event_swing_container.text.split('\n') if len(x) > 5]

    # get the odds changes
    odds_changes = [x for x in event_swing_container.text.split('\n') if '%' in x]

    # get the event name
    event_name = driver.find_element_by_class_name('table-header')
    # name is h1 in event_name
    name = event_name.find_element_by_tag_name('h1')
    event = name.text

    # get the url
    url = driver.current_url

    # create a dataframe
    df = pd.DataFrame({'names': names, 'odds_changes': odds_changes, 'event': event, 'event_url': url})
    
    #close the driver
    driver.close()

    return df

In [33]:
all_odds_eventlist = all_event_odds['event_odds_url'].unique().tolist()

In [34]:
already_complete = []
failed = []
newly_complete = []

for url in all_odds_eventlist:
    event_name = url.split('/')[-1]
    if  str(event_name) + '.csv' in os.listdir('data/final/odds/odds_changes/'):
        print('already done with ' + url.split('/')[-1])
        already_complete.append(event_name)
        continue

    else:
        try:
            df = get_bfodds(url)
            name = url.split('/')[-1]
            df.to_csv('data/final/odds/odds_changes/' + name + '.csv', index=False)
            newly_complete.append(name)
            print('done with ' + name)
        except:
            name = url.split('/')[-1]
            failed.append(name)
            print('failed on ' + name)
            continue
print("")
print('already done with ' + str(len(already_complete)) + ' events')
print('newly done with ' + str(len(newly_complete)) + ' events')
print('failed on ' + str(len(failed)) + ' events')

already done with ufc-100-137
already done with ufc-101-declaration-145
already done with ufc-102-couture-vs-nogueira-156
already done with ufc-103-franklin-vs-belfort-168
already done with ufc-105-couture-vs-vera-201
already done with ufc-106-ortiz-vs-griffin-2-209
already done with ufc-107-penn-vs-sanchez-189
already done with ufc-108-evans-vs-silva-219
already done with ufc-109-relentless-226
already done with ufc-110-nogueira-vs-velasquez-225
already done with ufc-111-st-pierre-vs-hardy-236
already done with ufc-112-invincible-245
already done with ufc-113-machida-vs-shogun-ii-247
already done with ufc-115-liddell-vs-franklin-255
already done with ufc-116-lesnar-vs-carwin-265
already done with ufc-118-edgar-vs-penn-2-278
already done with ufc-119-mir-vs-cro-cop-296
already done with ufc-120-bisping-vs-akiyama-294
already done with ufc-121-lesnar-vs-velasquez-308
already done with ufc-122-marquardt-vs-okami-323
already done with ufc-123-rampage-vs-machida-317
already done with ufc-1

In [35]:
# aggregate all odds changes into one dataframe
all_odds_changes = []

for file in os.listdir('data/final/odds/odds_changes/'):
    df = pd.read_csv('data/final/odds/odds_changes/' + file)
    # append
    all_odds_changes.append(df)

all_odds_changes = pd.concat(all_odds_changes)

print(all_odds_changes.shape)
all_odds_changes

(11423, 4)


,names,odds_changes,event,event_url
0,Deron Winn,-61%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
1,Chuck Liddell,-30%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
2,Gleison Tibau,-28%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
3,Ricardo Palacios,-14%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
4,Kenneth Bergh,+3%,GOLDEN BOY MMA: LIDDELL VS. ORTIZ 3,https://www.bestfightodds.com/events/golden-bo...
...,...,...,...,...
19,Daniel Santos,+34%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...
20,Brendan Allen,+35%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...
21,Mike Davis,+55%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...
22,Philipe Lins,+60%,UFC VEGAS 57,https://www.bestfightodds.com/events/ufc-vegas...


In [36]:
all_odds_changes.to_csv('data/final/aggregates/All_Odds_Changes_V1.csv', index=False)

### Edit Event_odds 

Get rid of alternate lines

In [37]:
all_event_odds = pd.read_csv('data/final/odds/All_Odds_by_Fighter_V2.csv')
all_event_odds

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
...,...,...,...,...,...,...,...
177994,Fight ends in TKO/KO/DQ in round 3,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
177995,Fight ends in submission in round 1,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
177996,Fight ends in submission in round 2,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
177997,Fight ends in submission in round 3,NaN,NaN,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan


In [38]:
all_options = all_event_odds['fighter'].value_counts()
all_options = pd.DataFrame(all_options)
all_options.reset_index(inplace=True)
delete_options = all_options.head(120)
delete_options

,index,fighter
0,Fight is not a draw,4643
1,Fight is a draw,4643
2,Fight starts round 3,4313
3,Fight won't start round 3,4313
4,Fight won't start round 2,4301
...,...,...
115,Johnson wins in round 3,58
116,Johnson wins in round 2,58
117,Johnson wins in round 1,57
118,Oliveira (scorecards = no action),51


In [39]:
delete_options_list = delete_options['index'].tolist()

In [40]:
# delete rows when fighter is in delete_options_list
all_event_odds = all_event_odds[~all_event_odds['fighter'].isin(delete_options_list)]
all_event_odds.head(2)

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100


In [41]:
# delete any rows where fighter has more than 3 words
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('round')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('decision')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('submission')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('knockout')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('draw')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('scorecards')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('majority')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('Not')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('wins')]
all_event_odds = all_event_odds[~all_event_odds['fighter'].str.contains('losses')]
all_event_odds


,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100
...,...,...,...,...,...,...,...
177793,Krzysztof Jotko,NaN,-124.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
177861,Chelsea Chandler,NaN,113.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
177862,Julija Stoliarenko,NaN,-133.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan
177930,Guido Cannetti,NaN,250.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan


# Add Odds Change to Odds

In [42]:

def get_odds_change_from_db(event_odds_url, fighter):
    
    # get the odds change from the all_odds_changes dataframe

    try:
        odds_change = all_odds_changes[(all_odds_changes['event_url'] == event_odds_url) & (all_odds_changes['names'] == fighter)]['odds_changes'].values[0]
        return odds_change
    except:
        return np.nan

In [43]:
# test
get_odds_change_from_db('https://www.bestfightodds.com/events/ufc-100-137', 'Brock Lesnar')

'-31%'

In [44]:
all_event_odds['odds_change'] = all_event_odds.apply(lambda row: get_odds_change_from_db(row['event_odds_url'], row['fighter']), axis=1)
all_event_odds

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name,odds_change
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-31%
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,+59%
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,NaN
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-6%
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,+4%
...,...,...,...,...,...,...,...,...
177793,Krzysztof Jotko,NaN,-124.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-32%
177861,Chelsea Chandler,NaN,113.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-6%
177862,Julija Stoliarenko,NaN,-133.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,+4%
177930,Guido Cannetti,NaN,250.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-22%


In [45]:
# get rid of %
all_event_odds['odds_change'] = all_event_odds['odds_change'].str.replace('%', '')
# turn into float
all_event_odds['odds_change'] = all_event_odds['odds_change'].astype(float)
all_event_odds.head(3)

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name,odds_change
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-31.0
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,59.0
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,NaN


In [46]:
# save to csv
all_event_odds.to_csv('data/final/odds/All_Odds_by_Fighter_WithChange.csv', index=False)

# Scraping UFC Stats

### Initial Scraping Functions

In [47]:
def get_event_date_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    date = soup.find('li', class_='b-list__box-list-item').text
    date = date.replace('\n', '')
    ed = date.find(':')
    date = date[ed+2:].strip()
    return date

In [48]:
get_event_date_from_ufcstats('http://www.ufcstats.com/event-details/93bf96be327fcd98')

'September 10, 2022'

In [49]:
def get_details_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    details = soup.find('div', class_='b-fight-details__content').text
    details = details.replace('\n', '')
    deet = details.find('Details:')
    details = details[deet+8:].strip()
    return details

In [50]:
get_details_from_ufcstats('http://www.ufcstats.com/fight-details/7e8fd03e070d0c25')

'Guillotine Choke From Bottom Guard'

In [51]:
def get_event_title_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('h2', class_='b-content__title').text
    title = title.replace('\n', '').strip()
    return title

In [52]:
get_event_title_from_ufcstats('http://www.ufcstats.com/fight-details/7e8fd03e070d0c25')

'UFC 279: Diaz vs. Ferguson'

In [53]:
def get_event_url_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('h2', class_='b-content__title')
    title = title.find('a')
    title = title['href']
    return title

In [54]:
get_event_url_from_ufcstats('http://www.ufcstats.com/fight-details/7e8fd03e070d0c25')

'http://www.ufcstats.com/event-details/93bf96be327fcd98'

In [55]:
def get_fight_urls(urls):

    # get links to all fights on each event page

    links = pd.DataFrame()
    for u in urls:
        try:
            reqs = requests.get(u)
            soup = BeautifulSoup(reqs.text, 'html.parser')
            title = soup.find('h2', class_='b-content__title').text
            title = title.replace('\n', '')
            date = soup.find('li', class_='b-list__box-list-item').text
            d = date.find('Date:')
            date = date[d+6:].replace('\n', '')
            for link in soup.find_all('a'):
                # append url and event title to df
                if 'fight-details' in link.get('href'):
                    links = links.append({'Fight_url': link.get('href'), 'Event_title': title, 'Date' : date}, ignore_index=True)

        except:
            return 'No Fighties'
            
    return links

In [56]:
# test get_fight_urls
get_fight_urls(['http://www.ufcstats.com/event-details/93bf96be327fcd98'])

,Fight_url,Event_title,Date
0,http://www.ufcstats.com/fight-details/7e8fd03e...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
1,http://www.ufcstats.com/fight-details/0111e6a6...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
2,http://www.ufcstats.com/fight-details/d3e23d7d...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
3,http://www.ufcstats.com/fight-details/989b8d36...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
4,http://www.ufcstats.com/fight-details/d869eaea...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
5,http://www.ufcstats.com/fight-details/0e6d6240...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
6,http://www.ufcstats.com/fight-details/301f230c...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
7,http://www.ufcstats.com/fight-details/77acb967...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
8,http://www.ufcstats.com/fight-details/bf334cb4...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
9,http://www.ufcstats.com/fight-details/db3e26d0...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"


In [57]:
def get_winner_from_ufcstats(url):

    ### This function scrapes the UFCStats website for the winner of a given fight###

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        both = soup.find_all('div', class_='b-fight-details__person')
        first = both[0].text.replace('   ', '').replace('\n', '').strip()
        second = both[1].text.replace('   ', '').replace('\n', '').strip()
        if(first.startswith('W')):
            winner = first[3:]
        else:
            winner = second[3:]
        return winner
        
    except:
        return nan

In [58]:
# test
get_winner_from_ufcstats('http://www.ufcstats.com/fight-details/7e8fd03e070d0c25')

'Nate Diaz'

In [59]:
def get_event_date_from_ufcstats(url):
    ### This function scrapes the UFCStats website for the event date of a given fight ###
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        date = soup.find('li', class_='b-list__box-list-item').text
        date = date.replace('\n', '')
        ed = date.find(':')
        date = date[ed+2:].strip()
        return date
    except:
        return nan

In [60]:
def get_event_details(event_id):
    # This function scrapes event details from ufc stats

    #try:
        df = pd.read_html('http://www.ufcstats.com/event-details/' + event_id)
        df = df[0]
        # split by space and get third element
        df['Fighter1'] = df['Fighter'].str.split('  ').str[0]
        df['Fighter2'] = df['Fighter'].str.split('  ').str[1]
        # fix Kd
        df['F1_Kd'] = df['Kd'].str.split('  ').str[0]
        df['F2_Kd'] = df['Kd'].str.split('  ').str[1]
        # fix Str
        df['F1_Str'] = df['Str'].str.split('  ').str[0]
        df['F2_Str'] = df['Str'].str.split('  ').str[1]
        # fix Td
        df['F1_Td'] = df['Td'].str.split('  ').str[0]
        df['F2_Td'] = df['Td'].str.split('  ').str[1]
        # fix Sub
        df['F1_Sub'] = df['Sub'].str.split('  ').str[0]
        df['F2_Sub'] = df['Sub'].str.split('  ').str[1]

        badcols = ['Fighter', 'Kd', 'Str', 'Td', 'Sub']

        # save index+1 to new column called "fight_num"
        df['fight_num'] = df.index + 1
        df['event_id'] = event_id

        # get fight links
        page = requests.get('http://www.ufcstats.com/event-details/' + event_id)
        soup = BeautifulSoup(page.content, 'html.parser')
        fight_linkies = soup.find_all('a')
        fight_links = [n['href'] for n in fight_linkies]
        fight_links = [n for n in fight_links if 'fight-details' in n]
        # delete duplicate links
        fight_links2 = pd.DataFrame(fight_links)
        fight_links2 = fight_links2.drop_duplicates()
        fight_links = fight_links2[0].tolist()

        # append fight links to df
        df['fight_link'] = fight_links

        df = df.drop(badcols, axis=1)
        df.to_csv('data/ufc_stats/events2/'+ event_id +'.csv')
        return df

In [61]:
def get_fight_totals(fight_details_url):
    # function returns the fight totals for a given fightid
    da_url = fight_details_url
    fightid = da_url[da_url.find('fight-details/')+14:]
    df = pd.read_html(da_url)
    # Part 1: Total Fight Stats
    totals = df[0]
    # replace '---' with 0
    for col in totals.columns:
        totals[col] = totals[col].astype(str).replace({'---': 0})
        totals[col] = totals[col].astype(str).replace({'--': 0})


    totals['Fighter_A'] = totals['Fighter'].str.split(' ').str[0] + ' ' + totals['Fighter'].str.split(' ').str[1]
    totals['Fighter_B'] = totals['Fighter'].str.split(' ').str[2] + ' ' + totals['Fighter'].str.split(' ').str[3]
    totals['A_Kd'] = totals['KD'].str.split('  ').str[0]
    totals['B_Kd'] = totals['KD'].str.split('  ').str[1]
    totals['A_Sig_strike'] = totals['Sig. str.'].str.split('  ').str[0]
    totals['B_Sig_strike'] = totals['Sig. str.'].str.split('  ').str[1]
    # split sig strikes into landed and attempted
    totals['A_Sig_strike_land'] = totals['A_Sig_strike'].str.split(' of ').str[0]
    totals['A_Sig_strike_att'] = totals['A_Sig_strike'].str.split(' of ').str[1]
    totals['B_Sig_strike_land'] = totals['B_Sig_strike'].str.split(' of ').str[0]
    totals['B_Sig_strike_att'] = totals['B_Sig_strike'].str.split(' of ').str[1]
    # change to numeric
    totals['A_Sig_strike_land'].replace({'---', 0}, inplace=True)
    totals['A_Sig_strike_att'].replace({'---', 0}, inplace=True)
    totals['B_Sig_strike_land'].replace({'---', 0}, inplace=True)
    totals['B_Sig_strike_att'].replace({'---', 0}, inplace=True)

    totals['A_Sig_strike_land'] = pd.to_numeric(totals['A_Sig_strike_land'])
    totals['A_Sig_strike_att'] = pd.to_numeric(totals['A_Sig_strike_att'])
    totals['B_Sig_strike_land'] = pd.to_numeric(totals['B_Sig_strike_land'])
    totals['B_Sig_strike_att'] = pd.to_numeric(totals['B_Sig_strike_att'])

    totals['A_Sig_strike_percent'] = totals['Sig. str. %'].str.split('  ').str[0]
    totals['B_Sig_strike_percent'] = totals['Sig. str. %'].str.split('  ').str[1]
    # get rid of % sign
    totals['A_Sig_strike_percent'] = totals['A_Sig_strike_percent'].str.replace('%', '')
    totals['B_Sig_strike_percent'] = totals['B_Sig_strike_percent'].str.replace('%', '')

    totals['A_Sig_strike_percent'] = totals['A_Sig_strike_percent'].astype(str).replace({'---': 0})
    totals['B_Sig_strike_percent'] = totals['B_Sig_strike_percent'].astype(str).replace({'---': 0})
    # change to numeric
    totals['A_Sig_strike_percent'] = pd.to_numeric(totals['A_Sig_strike_percent'])/100
    totals['B_Sig_strike_percent'] = pd.to_numeric(totals['B_Sig_strike_percent'])/100
    # total strikes
    totals['A_Total_Strikes'] = totals['Total str.'].str.split('  ').str[0]
    totals['B_Total_Strikes'] = totals['Total str.'].str.split('  ').str[1]
    # split total strikes into landed and attempted
    totals['A_Total_Strikes_land'] = totals['A_Total_Strikes'].str.split(' of ').str[0]
    totals['A_Total_Strikes_att'] = totals['A_Total_Strikes'].str.split(' of ').str[1]
    totals['B_Total_Strikes_land'] = totals['B_Total_Strikes'].str.split(' of ').str[0]
    totals['B_Total_Strikes_att'] = totals['B_Total_Strikes'].str.split(' of ').str[1]

    totals['A_Total_Strikes_land'].replace({'---',0}, inplace=True)
    totals['A_Total_Strikes_att'].replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_land'].replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_att'].replace({'---',0}, inplace=True)
    # change to numeric
    totals['A_Total_Strikes_land'] = pd.to_numeric(totals['A_Total_Strikes_land'])
    totals['A_Total_Strikes_att'] = pd.to_numeric(totals['A_Total_Strikes_att'])
    totals['B_Total_Strikes_land'] = pd.to_numeric(totals['B_Total_Strikes_land'])
    totals['B_Total_Strikes_att'] = pd.to_numeric(totals['B_Total_Strikes_att'])

    totals['A_Total_Strikes_land'].astype(str).replace({'---',0}, inplace=True)
    totals['A_Total_Strikes_att'].astype(str).replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_land'].astype(str).replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_att'].astype(str).replace({'---',0}, inplace=True) 

    # calculate total strike percentage
    totals['A_Total_Strikes_percent'] = totals['A_Total_Strikes_land'] / totals['A_Total_Strikes_att']
    totals['B_Total_Strikes_percent'] = totals['B_Total_Strikes_land'] / totals['B_Total_Strikes_att']
    # takedown stats
    totals['A_Takedowns'] = totals['Td'].str.split('  ').str[0]
    totals['B_Takedowns'] = totals['Td'].str.split('  ').str[1]
    # split takedowns into landed and attempted
    totals['A_Takedowns_land'] = totals['A_Takedowns'].str.split(' of ').str[0]
    totals['A_Takedowns_att'] = totals['A_Takedowns'].str.split(' of ').str[1]
    totals['B_Takedowns_land'] = totals['B_Takedowns'].str.split(' of ').str[0]
    totals['B_Takedowns_att'] = totals['B_Takedowns'].str.split(' of ').str[1]
    # change to numeric
    totals['A_Takedowns_land'] = pd.to_numeric(totals['A_Takedowns_land'])
    totals['A_Takedowns_att'] = pd.to_numeric(totals['A_Takedowns_att'])
    totals['B_Takedowns_land'] = pd.to_numeric(totals['B_Takedowns_land'])
    totals['B_Takedowns_att'] = pd.to_numeric(totals['B_Takedowns_att'])
    # fix %
    totals['Td %'].replace({'---': 0}, inplace=True)
    # get rid of ---
    totals['A_Takedowns_land'].replace({'---': 0}, inplace=True)
    totals['A_Takedowns_att'].replace({'---': 0}, inplace=True)
    totals['B_Takedowns_land'].replace({'---': 0}, inplace=True)
    totals['B_Takedowns_att'].replace({'---': 0}, inplace=True)
    totals['A_Takedown_percent'] = totals['A_Takedowns_land'] / totals['A_Takedowns_att']
    totals['B_Takedown_percent'] = totals['B_Takedowns_land'] / totals['B_Takedowns_att']
    # submission attempts
    totals['A_Sub_Attempts'] = totals['Sub. att'].str.split('  ').str[0]
    totals['B_Sub_Attempts'] = totals['Sub. att'].str.split('  ').str[1]
    # split submission attempts into landed and attempted
    totals['A_Sub_Attempts_land'] = totals['A_Sub_Attempts'].str.split(' of ').str[0]
    totals['A_Sub_Attempts_att'] = totals['A_Sub_Attempts'].str.split(' of ').str[1]
    totals['B_Sub_Attempts_land'] = totals['B_Sub_Attempts'].str.split(' of ').str[0]
    totals['B_Sub_Attempts_att'] = totals['B_Sub_Attempts'].str.split(' of ').str[1]
    # change to numeric
    totals['A_Sub_Attempts_land'] = pd.to_numeric(totals['A_Sub_Attempts_land'])
    totals['A_Sub_Attempts_att'] = pd.to_numeric(totals['A_Sub_Attempts_att'])
    totals['B_Sub_Attempts_land'] = pd.to_numeric(totals['B_Sub_Attempts_land'])
    totals['B_Sub_Attempts_att'] = pd.to_numeric(totals['B_Sub_Attempts_att'])
    # Rev
    totals['A_Rev'] = totals['Rev.'].str.split('  ').str[0]
    totals['B_Rev'] = totals['Rev.'].str.split('  ').str[1]
    # change to numeric
    totals['A_Rev'] = pd.to_numeric(totals['A_Rev'])
    totals['B_Rev'] = pd.to_numeric(totals['B_Rev'])
    # control time
    totals['A_Ctrl_time'] = totals['Ctrl'].str.split('  ').str[0]
    totals['B_Ctrl_time'] = totals['Ctrl'].str.split('  ').str[1]
    

    # split control time into minutes and seconds, and then convert to seconds
    totals['A_Ctrl_time_min'] = totals['A_Ctrl_time'].str.split(':').str[0]
    totals['A_Ctrl_time_sec'] = totals['A_Ctrl_time'].str.split(':').str[1]
    totals['B_Ctrl_time_min'] = totals['B_Ctrl_time'].str.split(':').str[0]
    totals['B_Ctrl_time_sec'] = totals['B_Ctrl_time'].str.split(':').str[1]

    totals['A_Ctrl_time_min'] = totals['A_Ctrl_time_min'].replace({'--': 0})
    totals['A_Ctrl_time_sec'] = totals['A_Ctrl_time_sec'].replace({'--': 0})
    totals['B_Ctrl_time_min'] = totals['B_Ctrl_time_min'].replace({'--': 0})
    totals['B_Ctrl_time_sec'] = totals['B_Ctrl_time_sec'].replace({'--': 0})

    # convert to seconds
    totals['A_Ctrl_time_tot'] = pd.to_numeric(totals['A_Ctrl_time_min']) * 60 + pd.to_numeric(totals['A_Ctrl_time_sec'])
    totals['B_Ctrl_time_tot'] = pd.to_numeric(totals['B_Ctrl_time_min']) * 60 + pd.to_numeric(totals['B_Ctrl_time_sec'])


    # add date with beatufil soup
    totals['details'] = get_details_from_ufcstats(da_url)
    totals['event_title'] = get_event_title_from_ufcstats(da_url)
    event_url = get_event_url_from_ufcstats(da_url)
    totals['event_url'] = event_url
    totals['date'] = get_event_date_from_ufcstats(event_url)

    try:
        totals['Winner']= get_winner_from_ufcstats(da_url)
    except:
        totals['Winner']= nan

    bad_cols = ['Fighter', 'KD', 'Sig. str. %', 'Sig. str.', 'Total str.', 'Td', 'Td %', 'Sub. att', 
                'Rev.', 'Ctrl', 'A_Sig_strike', 'B_Sig_strike', 'A_Takedowns', 'B_Takedowns',
                'A_Sub_Attempts', 'B_Sub_Attempts', 'A_Ctrl_time', 'B_Ctrl_time']
    totals.drop(columns=bad_cols, inplace=True)
    totals.to_csv('data/ufc_stats/fight_totals2/' + fightid + '_totals.csv')
    return totals
    


In [62]:
# test
get_fight_totals('http://www.ufcstats.com/fight-details/a419e6df765aa755')

,Fighter_A,Fighter_B,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,...,A_Ctrl_time_sec,B_Ctrl_time_min,B_Ctrl_time_sec,A_Ctrl_time_tot,B_Ctrl_time_tot,details,event_title,event_url,date,Winner
0,Stephen Thompson,Kevin Holland,0,0,163,260,119,210,0.62,0.56,...,47,1,21,47,81,to Corner Stoppage,UFC Fight Night: Thompson vs. Holland,http://www.ufcstats.com/event-details/b23388ff...,"December 03, 2022","Stephen Thompson ""Wonderboy"""


In [63]:
def get_significant_strikes(url):
    df = pd.read_html(url)
    # Part 2: Significant Strikes
    ss = df[2]
     
    ss['Fighter_A'] = ss['Fighter'].str.split(' ').str[0] + ' ' + ss['Fighter'].str.split(' ').str[1]
    ss['Fighter_B'] = ss['Fighter'].str.split(' ').str[2] + ' ' + ss['Fighter'].str.split(' ').str[3]
    ss['A_Head_Strikes'] = ss['Head'].str.split('  ').str[0]
    ss['B_Head_Strikes'] = ss['Head'].str.split('  ').str[1]
    # split head strikes into landed and attempted
    ss['A_Head_Strikes_land'] = ss['A_Head_Strikes'].str.split(' of ').str[0]
    ss['A_Head_Strikes_att'] = ss['A_Head_Strikes'].str.split(' of ').str[1]
    ss['B_Head_Strikes_land'] = ss['B_Head_Strikes'].str.split(' of ').str[0]
    ss['B_Head_Strikes_att'] = ss['B_Head_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Head_Strikes_land'] = pd.to_numeric(ss['A_Head_Strikes_land'])
    ss['A_Head_Strikes_att'] = pd.to_numeric(ss['A_Head_Strikes_att'])
    ss['B_Head_Strikes_land'] = pd.to_numeric(ss['B_Head_Strikes_land'])
    ss['B_Head_Strikes_att'] = pd.to_numeric(ss['B_Head_Strikes_att'])
    # Head Strikes Percentage
    ss['A_Head_Strikes_percent'] = ss['A_Head_Strikes_land'] / ss['A_Head_Strikes_att']
    ss['B_Head_Strikes_percent'] = ss['B_Head_Strikes_land'] / ss['B_Head_Strikes_att']
    # Body Strikes
    ss['A_Body_Strikes'] = ss['Body'].str.split('  ').str[0]
    ss['B_Body_Strikes'] = ss['Body'].str.split('  ').str[1]
    # split body strikes into landed and attempted
    ss['A_Body_Strikes_land'] = ss['A_Body_Strikes'].str.split(' of ').str[0]
    ss['A_Body_Strikes_att'] = ss['A_Body_Strikes'].str.split(' of ').str[1]
    ss['B_Body_Strikes_land'] = ss['B_Body_Strikes'].str.split(' of ').str[0]
    ss['B_Body_Strikes_att'] = ss['B_Body_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Body_Strikes_land'] = pd.to_numeric(ss['A_Body_Strikes_land'])
    ss['A_Body_Strikes_att'] = pd.to_numeric(ss['A_Body_Strikes_att'])
    ss['B_Body_Strikes_land'] = pd.to_numeric(ss['B_Body_Strikes_land'])
    ss['B_Body_Strikes_att'] = pd.to_numeric(ss['B_Body_Strikes_att'])
    # Body Strikes Percentage
    ss['A_Body_Strikes_percent'] = ss['A_Body_Strikes_land'] / ss['A_Body_Strikes_att']
    ss['B_Body_Strikes_percent'] = ss['B_Body_Strikes_land'] / ss['B_Body_Strikes_att']
    # Leg Strikes
    ss['A_Leg_Strikes'] = ss['Leg'].str.split('  ').str[0]
    ss['B_Leg_Strikes'] = ss['Leg'].str.split('  ').str[1]
    # split leg strikes into landed and attempted
    ss['A_Leg_Strikes_land'] = ss['A_Leg_Strikes'].str.split(' of ').str[0]
    ss['A_Leg_Strikes_att'] = ss['A_Leg_Strikes'].str.split(' of ').str[1]
    ss['B_Leg_Strikes_land'] = ss['B_Leg_Strikes'].str.split(' of ').str[0]
    ss['B_Leg_Strikes_att'] = ss['B_Leg_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Leg_Strikes_land'] = pd.to_numeric(ss['A_Leg_Strikes_land'])
    ss['A_Leg_Strikes_att'] = pd.to_numeric(ss['A_Leg_Strikes_att'])
    ss['B_Leg_Strikes_land'] = pd.to_numeric(ss['B_Leg_Strikes_land'])
    ss['B_Leg_Strikes_att'] = pd.to_numeric(ss['B_Leg_Strikes_att'])
    # Leg Strikes Percentage
    ss['A_Leg_Strikes_percent'] = ss['A_Leg_Strikes_land'] / ss['A_Leg_Strikes_att']
    ss['B_Leg_Strikes_percent'] = ss['B_Leg_Strikes_land'] / ss['B_Leg_Strikes_att']
    # Distance Strikes
    ss['A_Distance_Strikes'] = ss['Distance'].str.split('  ').str[0]
    ss['B_Distance_Strikes'] = ss['Distance'].str.split('  ').str[1]
    # split distance strikes into landed and attempted
    ss['A_Distance_Strikes_land'] = ss['A_Distance_Strikes'].str.split(' of ').str[0]
    ss['A_Distance_Strikes_att'] = ss['A_Distance_Strikes'].str.split(' of ').str[1]
    ss['B_Distance_Strikes_land'] = ss['B_Distance_Strikes'].str.split(' of ').str[0]
    ss['B_Distance_Strikes_att'] = ss['B_Distance_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Distance_Strikes_land'] = pd.to_numeric(ss['A_Distance_Strikes_land'])
    ss['A_Distance_Strikes_att'] = pd.to_numeric(ss['A_Distance_Strikes_att'])
    ss['B_Distance_Strikes_land'] = pd.to_numeric(ss['B_Distance_Strikes_land'])
    ss['B_Distance_Strikes_att'] = pd.to_numeric(ss['B_Distance_Strikes_att'])
    # distance strikes percentage
    ss['A_Distance_Strikes_percent'] = ss['A_Distance_Strikes_land'] / ss['A_Distance_Strikes_att']
    ss['B_Distance_Strikes_percent'] = ss['B_Distance_Strikes_land'] / ss['B_Distance_Strikes_att']

    # Clinch Strikes
    ss['A_Clintch_Strikes'] = ss['Clinch'].str.split('  ').str[0]
    ss['B_Clintch_Strikes'] = ss['Clinch'].str.split('  ').str[1]
    # split clinch strikes into landed and attempted
    ss['A_Clinch_Strikes_land'] = ss['A_Clintch_Strikes'].str.split(' of ').str[0]
    ss['A_Clinch_Strikes_att'] = ss['A_Clintch_Strikes'].str.split(' of ').str[1]
    ss['B_Clinch_Strikes_land'] = ss['B_Clintch_Strikes'].str.split(' of ').str[0]
    ss['B_Clinch_Strikes_att'] = ss['B_Clintch_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Clinch_Strikes_land'] = pd.to_numeric(ss['A_Clinch_Strikes_land'])
    ss['A_Clinch_Strikes_att'] = pd.to_numeric(ss['A_Clinch_Strikes_att'])
    ss['B_Clinch_Strikes_land'] = pd.to_numeric(ss['B_Clinch_Strikes_land'])
    ss['B_Clinch_Strikes_att'] = pd.to_numeric(ss['B_Clinch_Strikes_att'])
    # clinch strikes percentage
    ss['A_Clinch_Strikes_percent'] = ss['A_Clinch_Strikes_land'] / ss['A_Clinch_Strikes_att']
    ss['B_Clinch_Strikes_percent'] = ss['B_Clinch_Strikes_land'] / ss['B_Clinch_Strikes_att']

    # Ground Strikes
    ss['A_Ground_Strikes'] = ss['Ground'].str.split('  ').str[0]
    ss['B_Ground_Strikes'] = ss['Ground'].str.split('  ').str[1]
    # split ground strikes into landed and attempted
    ss['A_Ground_Strikes_land'] = ss['A_Ground_Strikes'].str.split(' of ').str[0]
    ss['A_Ground_Strikes_att'] = ss['A_Ground_Strikes'].str.split(' of ').str[1]
    ss['B_Ground_Strikes_land'] = ss['B_Ground_Strikes'].str.split(' of ').str[0]
    ss['B_Ground_Strikes_att'] = ss['B_Ground_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Ground_Strikes_land'] = pd.to_numeric(ss['A_Ground_Strikes_land'])
    ss['A_Ground_Strikes_att'] = pd.to_numeric(ss['A_Ground_Strikes_att'])
    ss['B_Ground_Strikes_land'] = pd.to_numeric(ss['B_Ground_Strikes_land'])
    ss['B_Ground_Strikes_att'] = pd.to_numeric(ss['B_Ground_Strikes_att'])
    # ground strikes percentage
    ss['A_Ground_Strikes_percent'] = ss['A_Ground_Strikes_land'] / ss['A_Ground_Strikes_att']
    ss['B_Ground_Strikes_percent'] = ss['B_Ground_Strikes_land'] / ss['B_Ground_Strikes_att']
    # add stuff with beautiful soup
    ss['details'] = get_details_from_ufcstats(url)
    ss['event_title'] = get_event_title_from_ufcstats(url)
    event_url = get_event_url_from_ufcstats(url)
    ss['event_url'] = event_url

    fightid = url[url.find('fight-details/')+14:]

    bad_cols = ['Fighter', 'Sig. str', 'Sig. str. %', 'Head', 'Body', 'Leg', 'Distance', 'Clinch', 'Ground',
                'A_Head_Strikes', 'B_Head_Strikes', 'A_Body_Strikes', 'B_Body_Strikes', 'A_Leg_Strikes', 'B_Leg_Strikes',
                'A_Distance_Strikes', 'B_Distance_Strikes', 'A_Clintch_Strikes', 'B_Clintch_Strikes', 'A_Ground_Strikes', 
                'B_Ground_Strikes']
    ss.drop(columns=bad_cols, inplace=True)
    ss.to_csv('data/ufc_stats/sig_strikes2/' + fightid + '_sigstrikes.csv')

    return ss

In [64]:
# get all events from oddsbyfighter
all_event_odds = pd.read_csv('data/final/odds/All_Odds_by_Fighter_WithChange.csv')
all_event_odds

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name,odds_change
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-31.0
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,59.0
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,NaN
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-6.0
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,4.0
...,...,...,...,...,...,...,...,...
22949,Krzysztof Jotko,NaN,-124.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-32.0
22950,Chelsea Chandler,NaN,113.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-6.0
22951,Julija Stoliarenko,NaN,-133.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,4.0
22952,Guido Cannetti,NaN,250.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-22.0


In [65]:
# get all events from all_event_odds

all_event_odds_urls = all_event_odds['event_ufcstats_url'].unique()
print(len(all_event_odds_urls))
all_event_odds_urls

490


array(['http://ufcstats.com/event-details/0ee783aa00e468f0',
       'http://ufcstats.com/event-details/48d1f690b763934c',
       'http://ufcstats.com/event-details/c6a33ff198aaaeeb',
       'http://ufcstats.com/event-details/896c322f56b8be5a',
       'http://ufcstats.com/event-details/fa8b9e6b0c2269f8',
       'http://ufcstats.com/event-details/84a067c46306a737',
       'http://ufcstats.com/event-details/821cd80aab70d5f9',
       'http://ufcstats.com/event-details/a6d8bfe9e0c8153b',
       'http://ufcstats.com/event-details/15edcf67ccf5be84',
       'http://ufcstats.com/event-details/49efbdc6c9f650c4',
       'http://ufcstats.com/event-details/c80095f6092271a7',
       'http://ufcstats.com/event-details/24b033b3daf1c9df',
       'http://ufcstats.com/event-details/d0c29452d3272603',
       'http://ufcstats.com/event-details/cfb65863d5099327',
       'http://ufcstats.com/event-details/140745cbbcb023ac',
       'http://ufcstats.com/event-details/054defd5420a551f',
       'http://ufcstats.

In [66]:
all_event_urls = all_event_odds_urls.tolist()

In [67]:
# get all the fight links from an event page, dl to events folder
events_with_fight_links = pd.DataFrame()
m = 1
errors = []

for event_url in all_event_urls:
    try:
        event_id = event_url.split('/')[-1]
        get_event_details(event_id)
        print(f'{m} of {len(all_event_urls)}')
        m += 1
    
    except:
        errors.append(event_url)
        print(f'ERROR: {m} of {len(all_event_urls)}')
        m += 1

print(len(errors))
errors


1 of 490
2 of 490
3 of 490
4 of 490
5 of 490
6 of 490
7 of 490
8 of 490
9 of 490
10 of 490
11 of 490
12 of 490
13 of 490
14 of 490
15 of 490
16 of 490
17 of 490
18 of 490
19 of 490
20 of 490
21 of 490
22 of 490
23 of 490
24 of 490
25 of 490
26 of 490
27 of 490
28 of 490
29 of 490
30 of 490
31 of 490
32 of 490
33 of 490
34 of 490
35 of 490
36 of 490
37 of 490
38 of 490
39 of 490
40 of 490
41 of 490
42 of 490
43 of 490
44 of 490
45 of 490
46 of 490
47 of 490
48 of 490
49 of 490
50 of 490
51 of 490
52 of 490
53 of 490
54 of 490
55 of 490
56 of 490
57 of 490
58 of 490
59 of 490
60 of 490
61 of 490
62 of 490
63 of 490
64 of 490
65 of 490
66 of 490
67 of 490
68 of 490
69 of 490
70 of 490
71 of 490
72 of 490
73 of 490
74 of 490
75 of 490
76 of 490
77 of 490
78 of 490
79 of 490
80 of 490
81 of 490
82 of 490
83 of 490
84 of 490
85 of 490
86 of 490
87 of 490
88 of 490
89 of 490
90 of 490
91 of 490
92 of 490
93 of 490
94 of 490
95 of 490
96 of 490
97 of 490
98 of 490
99 of 490
100 of 490
101 of 4

[nan]

In [68]:
# Aggregate all the fight data from all the events
all_event_data = pd.DataFrame()
m = 1
errors = []

event_files = os.listdir('data/ufc_stats/events2/')

for event_file in event_files:
    data = pd.read_csv('data/ufc_stats/events2/' + event_file)
    all_event_data = all_event_data.append(data, ignore_index=True)

all_event_data

,Unnamed: 0,W/L,Weight class,Method,Round,Time,Fighter1,Fighter2,F1_Kd,F2_Kd,F1_Str,F2_Str,F1_Td,F2_Td,F1_Sub,F2_Sub,fight_num,event_id,fight_link
0,0,win,Heavyweight,KO/TKO Punches,3,4:23,Ciryl Gane,Tai Tuivasa,1,1,110,29,0,0,0,0,1,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/810c9da0...
1,1,win,Middleweight,U-DEC,3,5:00,Robert Whittaker,Marvin Vettori,0,0,74,33,1,0,0,0,2,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/b8ca1acd...
2,2,win,Middleweight,U-DEC,3,5:00,Nassourdine Imavov,Joaquin Buckley,0,0,51,46,2,0,2,0,3,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/20f316f9...
3,3,win,Middleweight,KO/TKO Punches,3,1:09,Roman Kopylov,Alessio Di Chirico,1,0,59,39,0,0,0,0,4,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/1a191251...
4,4,win,Featherweight,U-DEC,3,5:00,William Gomis,Jarno Errens,0,0,32,20,3,0,0,1,5,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/398db400...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5673,7,win,Featherweight,SUB Guillotine Choke,3,2:59,Brian Ortega,Renato Moicano,0,0,65,109,0,2,1,0,8,ff9578cdbfabd323,http://www.ufcstats.com/fight-details/4924d7a9...
5674,8,win,Featherweight,U-DEC,3,5:00,Calvin Kattar,Andre Fili,0,0,75,73,2,0,0,0,9,ff9578cdbfabd323,http://www.ufcstats.com/fight-details/3d95b4f9...
5675,9,win,Women's Strawweight,U-DEC,3,5:00,Aleksandra Albu,Kailin Curran,0,0,82,86,5,1,0,0,10,ff9578cdbfabd323,http://www.ufcstats.com/fight-details/e80a8338...
5676,10,win,Flyweight,S-DEC,3,5:00,Jarred Brooks,Eric Shelton,0,0,27,31,5,1,1,1,11,ff9578cdbfabd323,http://www.ufcstats.com/fight-details/4f480695...


In [69]:
all_event_data.to_csv('data/final/events/All_Events_Fights_and_FightUrls.csv')

# 3. Merge UFC Stats Data with Event_Odds

In [70]:
# Make master fight_totals dataframe

fight_totals_files = os.listdir('data/ufc_stats/fight_totals2')
sig_strike_files = os.listdir('data/ufc_stats/sig_strikes2')


In [71]:
# Make master fight_totals dataframe
fight_totals = pd.DataFrame()

for file in fight_totals_files: 
    file2 = file.replace('.csv','')
    df = pd.read_csv('data/ufc_stats/fight_totals2/'+file)
    # add fight id column
    df['fight_id'] = file2[:-7]
    fight_totals = fight_totals.append(df)
fight_totals = fight_totals.reset_index(drop=True)

fight_totals

,Unnamed: 0,Fighter_A,Fighter_B,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,...,B_Ctrl_time_min,B_Ctrl_time_sec,A_Ctrl_time_tot,B_Ctrl_time_tot,details,event_title,event_url,date,Winner,fight_id
0,0,Holly Holm,Irene Aldana,0,0,154,301,69,185,0.51,...,0,3.0,316.0,3.0,Mike Bell ...,UFC Fight Night: Holm vs. Aldana,http://www.ufcstats.com/event-details/805ad180...,"October 03, 2020","Holly Holm ""The Preacher's Daughter""",0005e00b07cee542
1,0,Joshua Burkman,Josh Neer,1,0,35,88,19,39,0.39,...,3,10.0,405.0,190.0,Abe Belardo ...,UFC 61: Bitter Rivals,http://www.ufcstats.com/event-details/f70144ca...,"July 08, 2006","Joshua Burkman ""The People's Warrior""",000da3152b7b5ab1
2,0,Paddy Pimblett,Kazula Vargas,0,0,3,6,7,9,0.50,...,2,43.0,53.0,163.0,Rear Naked Choke,UFC Fight Night: Volkov vs. Aspinall,http://www.ufcstats.com/event-details/1d007568...,"March 19, 2022","Paddy Pimblett ""The Baddy""",001441f70c293931
3,0,Greg Hardy,Ben Sosoli,0,0,54,105,26,97,0.51,...,0,0.0,0.0,0.0,Illegal Inhaler Use by Hardy La...,UFC Fight Night: Reyes vs. Weidman,http://www.ufcstats.com/event-details/3ae10ac4...,"October 18, 2019","Ben Sosoli ""Combat Wombat""",0019ec81fd706ade
4,0,Jared Rosholt,Josh Copeland,0,0,22,45,9,36,0.48,...,0,0.0,319.0,0.0,Punches to Head From Side Control,UFC 185: Pettis vs Dos Anjos,http://www.ufcstats.com/event-details/f54200f1...,"March 14, 2015",Jared Rosholt,0027e179b743c86c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8434,0,Tim Sylvia,Wesley Correira,0,0,138,214,26,65,0.64,...,0,0.0,3.0,0.0,to Corner Stoppage,UFC 39: The Warriors Return,http://www.ufcstats.com/event-details/0cf93551...,"September 27, 2002","Tim Sylvia ""The Maine-iac""",ffe4379d6bd1e82b
8435,0,Kimo Leopoldo,David Abbott,0,0,1,1,0,0,1.00,...,0,1.0,112.0,1.0,Arm Triangle From Mount,UFC 43: Meltdown,http://www.ufcstats.com/event-details/108afe61...,"June 06, 2003",Kimo Leopoldo,ffe629a5232a878b
8436,0,Tony Ferguson,Gleison Tibau,0,0,11,38,2,20,0.28,...,0,0.0,45.0,0.0,Rear Naked Choke,UFC 184: Rousey vs Zingano,http://www.ufcstats.com/event-details/ad4e9055...,"February 28, 2015","Tony Ferguson ""El Cucuy""",ffea776913451b6d
8437,0,Tim Boetsch,CB Dollaway,0,0,41,76,57,118,0.53,...,3,57.0,119.0,237.0,Point Deducted: Eye Pokes by Dollaway ...,UFC 166: Velasquez vs Dos Santos 3,http://www.ufcstats.com/event-details/eae4aec1...,"October 19, 2013","Tim Boetsch ""The Barbarian""",fffa21388cdd78b7


In [72]:
sig_strike_agg = pd.DataFrame()
for file in sig_strike_files:
    file2 = file.replace('.csv','')
    df = pd.read_csv('data/ufc_stats/sig_strikes2/'+file)
    df['fight_id'] = file2[:-11]
    sig_strike_agg = sig_strike_agg.append(df)
sig_strike_agg = sig_strike_agg.reset_index(drop=True)

sig_strike_agg

,Unnamed: 0,Fighter_A,Fighter_B,A_Head_Strikes_land,A_Head_Strikes_att,B_Head_Strikes_land,B_Head_Strikes_att,A_Head_Strikes_percent,B_Head_Strikes_percent,A_Body_Strikes_land,...,A_Ground_Strikes_land,A_Ground_Strikes_att,B_Ground_Strikes_land,B_Ground_Strikes_att,A_Ground_Strikes_percent,B_Ground_Strikes_percent,details,event_title,event_url,fight_id
0,0,Holly Holm,Irene Aldana,81,214,38,128,0.378505,0.296875,56,...,13,28,0,0,0.464286,NaN,Mike Bell ...,UFC Fight Night: Holm vs. Aldana,http://www.ufcstats.com/event-details/805ad180...,0005e00b07cee542
1,0,Joshua Burkman,Josh Neer,22,72,17,36,0.305556,0.472222,9,...,5,10,7,9,0.500000,0.777778,Abe Belardo ...,UFC 61: Bitter Rivals,http://www.ufcstats.com/event-details/f70144ca...,000da3152b7b5ab1
2,0,Paddy Pimblett,Kazula Vargas,1,4,5,7,0.250000,0.714286,0,...,1,1,3,3,1.000000,1.000000,Rear Naked Choke,UFC Fight Night: Volkov vs. Aspinall,http://www.ufcstats.com/event-details/1d007568...,001441f70c293931
3,0,Greg Hardy,Ben Sosoli,22,72,15,78,0.305556,0.192308,5,...,0,0,0,0,NaN,NaN,Illegal Inhaler Use by Hardy La...,UFC Fight Night: Reyes vs. Weidman,http://www.ufcstats.com/event-details/3ae10ac4...,0019ec81fd706ade
4,0,Jared Rosholt,Josh Copeland,19,41,7,34,0.463415,0.205882,3,...,10,17,0,0,0.588235,NaN,Punches to Head From Side Control,UFC 185: Pettis vs Dos Anjos,http://www.ufcstats.com/event-details/f54200f1...,0027e179b743c86c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8434,0,Tim Sylvia,Wesley Correira,115,186,21,60,0.618280,0.350000,13,...,0,0,0,0,NaN,NaN,to Corner Stoppage,UFC 39: The Warriors Return,http://www.ufcstats.com/event-details/0cf93551...,ffe4379d6bd1e82b
8435,0,Kimo Leopoldo,David Abbott,0,0,0,0,NaN,NaN,1,...,0,0,0,0,NaN,NaN,Arm Triangle From Mount,UFC 43: Meltdown,http://www.ufcstats.com/event-details/108afe61...,ffe629a5232a878b
8436,0,Tony Ferguson,Gleison Tibau,5,27,2,18,0.185185,0.111111,3,...,3,8,0,0,0.375000,NaN,Rear Naked Choke,UFC 184: Rousey vs Zingano,http://www.ufcstats.com/event-details/ad4e9055...,ffea776913451b6d
8437,0,Tim Boetsch,CB Dollaway,27,59,33,92,0.457627,0.358696,4,...,2,2,3,5,1.000000,0.600000,Point Deducted: Eye Pokes by Dollaway ...,UFC 166: Velasquez vs Dos Santos 3,http://www.ufcstats.com/event-details/eae4aec1...,fffa21388cdd78b7


In [73]:
fight_totals.to_csv('data/final/aggregates/All_Fight_Totals.csv', index=False)
sig_strike_agg.to_csv('data/final/aggregates/All_Sig_Strikes.csv', index=False)

In [74]:
# get the columns that are in both dataframes
common_cols = list(set(fight_totals.columns).intersection(sig_strike_agg.columns))
common_cols

['event_title',
 'Fighter_B',
 'event_url',
 'Unnamed: 0',
 'Fighter_A',
 'fight_id',
 'details']

In [75]:
# drop unnamed
fight_totals = fight_totals.drop(columns=['Unnamed: 0'])
sig_strike_agg = sig_strike_agg.drop(columns=['Unnamed: 0'])

In [76]:
# drop details from sig_strike_agg
sig_strike_agg = sig_strike_agg.drop(columns=['details'])

In [77]:
common_cols = list(set(fight_totals.columns).intersection(sig_strike_agg.columns))
common_cols

['event_title', 'Fighter_B', 'event_url', 'Fighter_A', 'fight_id']

In [78]:
# merge the two dataframes, using the common columns as the key
fight_DF = pd.merge(fight_totals, sig_strike_agg, on=common_cols, how= 'inner')
fight_DF

,Fighter_A,Fighter_B,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,...,B_Clinch_Strikes_land,B_Clinch_Strikes_att,A_Clinch_Strikes_percent,B_Clinch_Strikes_percent,A_Ground_Strikes_land,A_Ground_Strikes_att,B_Ground_Strikes_land,B_Ground_Strikes_att,A_Ground_Strikes_percent,B_Ground_Strikes_percent
0,Holly Holm,Irene Aldana,0,0,154,301,69,185,0.51,0.37,...,4,4,1.000000,1.000000,13,28,0,0,0.464286,NaN
1,Joshua Burkman,Josh Neer,1,0,35,88,19,39,0.39,0.48,...,5,9,0.571429,0.555556,5,10,7,9,0.500000,0.777778
2,Paddy Pimblett,Kazula Vargas,0,0,3,6,7,9,0.50,0.77,...,2,2,0.000000,1.000000,1,1,3,3,1.000000,1.000000
3,Greg Hardy,Ben Sosoli,0,0,54,105,26,97,0.51,0.26,...,0,1,1.000000,0.000000,0,0,0,0,NaN,NaN
4,Jared Rosholt,Josh Copeland,0,0,22,45,9,36,0.48,0.25,...,3,8,0.750000,0.375000,10,17,0,0,0.588235,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8432,Tim Sylvia,Wesley Correira,0,0,138,214,26,65,0.64,0.40,...,2,3,0.796610,0.666667,0,0,0,0,NaN,NaN
8433,Kimo Leopoldo,David Abbott,0,0,1,1,0,0,1.00,0.00,...,0,0,NaN,NaN,0,0,0,0,NaN,NaN
8434,Tony Ferguson,Gleison Tibau,0,0,11,38,2,20,0.28,0.10,...,0,0,NaN,NaN,3,8,0,0,0.375000,NaN
8435,Tim Boetsch,CB Dollaway,0,0,41,76,57,118,0.53,0.48,...,13,21,0.666667,0.619048,2,2,3,5,1.000000,0.600000


In [79]:
# check for nulls
fight_DF.isnull().sum()

Fighter_A                      0
Fighter_B                     21
A_Kd                           0
B_Kd                           0
A_Sig_strike_land              0
                            ... 
A_Ground_Strikes_att           0
B_Ground_Strikes_land          0
B_Ground_Strikes_att           0
A_Ground_Strikes_percent    2668
B_Ground_Strikes_percent    3818
Length: 78, dtype: int64

In [80]:
# drop when Fighter_B is null
fight_DF = fight_DF.dropna(subset=['Fighter_B'])

In [81]:
missing = fight_DF.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing

A_Total_Strikes_percent         37
Winner                          57
B_Total_Strikes_percent         65
A_Head_Strikes_percent          96
B_Distance_Strikes_percent     169
A_Distance_Strikes_percent     172
B_Head_Strikes_percent         182
details                        282
A_Body_Strikes_percent        1017
A_Ctrl_time_sec               1211
B_Ctrl_time_sec               1211
A_Ctrl_time_tot               1211
B_Ctrl_time_tot               1211
B_Body_Strikes_percent        1268
A_Leg_Strikes_percent         1821
B_Leg_Strikes_percent         1864
A_Clinch_Strikes_percent      2044
B_Clinch_Strikes_percent      2122
A_Ground_Strikes_percent      2661
A_Takedown_percent            2705
B_Takedown_percent            3201
B_Ground_Strikes_percent      3811
A_Sub_Attempts_att            8416
B_Sub_Attempts_att            8416
dtype: int64

In [82]:
# drop when Winner is missing
fight_DF = fight_DF.dropna(subset=['Winner'])

In [83]:
# change all the nulls to 0
fight_DF = fight_DF.fillna(0)

In [84]:
fight_DF.to_csv('data/final/aggregates/Fight_DF.csv', index=False)

##### This is the data set that will be the starting point in our feature engineering and modeling. From here on out, we are just 

## Scraping Fighter Biography Infro from UFC.Com

Can do via URL:

Example: https://www.ufc.com/athlete/kevin-holland

- All: https://www.ufc.com/athletes/all/active

#### List all Fighters

In [85]:
df = pd.read_csv('data/final/aggregates/All_Fight_Totals.csv')

In [86]:
# list of all fighter_A and fighter_B names
fighter_names = list(df['Fighter_A'].unique()) + list(df['Fighter_B'].unique())

# remove duplicates
fighter_names = list(set(fighter_names))

len(fighter_names)

3103

In [87]:
fighter_name_df = pd.DataFrame(fighter_names, columns = ['Fighter_Name'])
fighter_name_df

,Fighter_Name
0,Nasrat Haqparast
1,Mickael Lebout
2,Tom Niinimaki
3,Michael Johnson
4,Tony Gravely
...,...
3098,Jorge Lopez
3099,Rodrigo de
3100,Aaron Jeffery
3101,William Macario


In [88]:
fighter_name_df['ufc_url'] = 'https://www.ufc.com/athlete/' + fighter_name_df['Fighter_Name'].str.lower().str.replace(' ', '-')
fighter_name_df.head()

,Fighter_Name,ufc_url
0,Nasrat Haqparast,https://www.ufc.com/athlete/nasrat-haqparast
1,Mickael Lebout,https://www.ufc.com/athlete/mickael-lebout
2,Tom Niinimaki,https://www.ufc.com/athlete/tom-niinimaki
3,Michael Johnson,https://www.ufc.com/athlete/michael-johnson
4,Tony Gravely,https://www.ufc.com/athlete/tony-gravely


#### Function to scrape fighter bio information from UFC.com

In [89]:
def get_ufc_bios(url):
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        bio = soup.find('div', class_='c-bio__info-details').text

        fighter = soup.find('h1', class_='hero-profile__name').text

        # get every c-bio__label and c-bio__text within bio
        labels = soup.find_all('div', class_='c-bio__label')
        texts = soup.find_all('div', class_='c-bio__text')

        # create empty lists to store labels and texts
        label_list = []
        text_list = []

        # loop through labels and texts and append to lists
        for label in labels:
            label_list.append(label.text)

        for text in texts:
            text_list.append(text.text)

        # create dataframe from lists, with label as column names
        fighter_bio_df = pd.DataFrame([label_list, text_list])
        fighter_bio_df.columns = fighter_bio_df.iloc[0]

        # drop 1st row
        fighter_bio_df = fighter_bio_df.drop(fighter_bio_df.index[0])



        # add fighter name column
        fighter_bio_df['fighter'] = fighter

        # replace any '\n' with ''
        fighter_bio_df = fighter_bio_df.replace('\n', '', regex=True)

        fighter_bio_df.to_csv('data/final/fighters/' + fighter + '.csv')
        return fighter_bio_df

    except:
        # If there is an error, save it in the bios with errors folder

        print('Error with ' + str(url))
        data = {'fighter': [url]}
        df = pd.DataFrame(data)
        url2 = url.replace('https://www.ufc.com/athlete/', '')
        url3 = url2.replace('-', ' ')
        df.to_csv('data/final/fighters/bios_with_errors/' + url2 + '.csv')


In [90]:
# test
get_ufc_bios('https://www.ufc.com/athlete/Max-Holloway')

,Status,Place of Birth,Trains at,Fighting style,Age,Height,Weight,Octagon Debut,Reach,Leg reach,fighter
1,Active,"Waianae, United States",Hawaii Elite MMA - Hawaii,Muay Thai,30,71.00,144.50,"Feb. 04, 2012",69.00,42.00,Max Holloway


##### Check if fighter bio data has already been scraped. If not, scrape it. 

In [91]:
def download_missing_bios():
    working_bios_folder = os.listdir('data/final/fighters/')
    working_bios = [x[:-4] for x in working_bios_folder]

    bios_with_errors_folder = os.listdir('data/final/fighters/bios_with_errors/')
    bios_with_errors = [x[:-4] for x in bios_with_errors_folder]

    all_downloaded_bios = working_bios + bios_with_errors
    un_downloaded_bios = fighter_name_df[~fighter_name_df['Fighter_Name'].isin(all_downloaded_bios)]

    # delete any NAN in undownload bios
    un_downloaded_bios = un_downloaded_bios.dropna()

    # download all undownloaded bios
    i = 0
    fighters = len(un_downloaded_bios['ufc_url'])

    # Downloading data for ALL fighters

    for url in un_downloaded_bios['ufc_url']:
        get_ufc_bios(url)
        print('Done with ' + url + ' ' + str(i) + ' of ' + str(fighters))
        i += 1


In [92]:
download_missing_bios()

Error with https://www.ufc.com/athlete/aoriqileng-jay
Done with https://www.ufc.com/athlete/aoriqileng-jay 0 of 721
Error with https://www.ufc.com/athlete/emmanuel-yarborough
Done with https://www.ufc.com/athlete/emmanuel-yarborough 1 of 721
Error with https://www.ufc.com/athlete/de-andrade
Done with https://www.ufc.com/athlete/de-andrade 2 of 721
Error with https://www.ufc.com/athlete/ovince-saint
Done with https://www.ufc.com/athlete/ovince-saint 3 of 721
Error with https://www.ufc.com/athlete/robert-mcdaniel
Done with https://www.ufc.com/athlete/robert-mcdaniel 4 of 721
Error with https://www.ufc.com/athlete/joseph-gigliotti
Done with https://www.ufc.com/athlete/joseph-gigliotti 5 of 721
Error with https://www.ufc.com/athlete/aurelijus-kerpe
Done with https://www.ufc.com/athlete/aurelijus-kerpe 6 of 721
Error with https://www.ufc.com/athlete/devin-cole
Done with https://www.ufc.com/athlete/devin-cole 7 of 721
Error with https://www.ufc.com/athlete/huna-nate
Done with https://www.ufc

In [93]:
len(os.listdir('data/final/fighters/'))

2419

In [94]:
len(os.listdir('data/final/fighters/bios_with_errors/'))

686

##### There are many bios with errors, but none for important fighters who have fought in the last 5 years.

# MasterDF for fighter bios

Here, I combine all the bio data into one dataframe.

In [95]:
fighter_bio_files=  os.listdir('data/final/fighters/')
fighter_bio_files = [x for x in fighter_bio_files if 'csv' in x]

# append all fighter bios into one dataframe
fighter_bio_df = pd.DataFrame()
for file in fighter_bio_files:
    df = pd.read_csv('data/final/fighters/' + file)
    fighter_bio_df = fighter_bio_df.append(df)

fighter_bio_df = fighter_bio_df.reset_index(drop=True)
fighter_bio_df.head()

,Unnamed: 0,Status,Place of Birth,Fighting style,Age,Height,Weight,Octagon Debut,Reach,Leg reach,fighter,Trains at
0,1,Not Fighting,"Parrish, United States",MMA,32.0,72.0,155.0,"Jul. 30, 2019",78.0,42.0,Aalon Cruz,NaN
1,1,Not Fighting,"Newport Beach, United States",NaN,47.0,75.0,231.0,"Nov. 17, 2000",NaN,NaN,Aaron Brink,NaN
2,1,NaN,"Tillsonburg, Canada",MMA,29.0,74.0,185.5,"Dec. 11, 2022",73.5,41.0,Aaron Jeffery,NaN
3,1,Active,"Houston, United States",Muay Thai,32.0,69.0,135.0,"May. 24, 2014",71.0,40.0,Aaron Phillips,Headkicks MMA
4,1,Not Fighting,"Tell City, United States",NaN,40.0,68.0,156.0,"May. 10, 2002",NaN,NaN,Aaron Riley,NaN


In [96]:
fighter_bio_df.head(3)

,Unnamed: 0,Status,Place of Birth,Fighting style,Age,Height,Weight,Octagon Debut,Reach,Leg reach,fighter,Trains at
0,1,Not Fighting,"Parrish, United States",MMA,32.0,72.0,155.0,"Jul. 30, 2019",78.0,42.0,Aalon Cruz,NaN
1,1,Not Fighting,"Newport Beach, United States",NaN,47.0,75.0,231.0,"Nov. 17, 2000",NaN,NaN,Aaron Brink,NaN
2,1,NaN,"Tillsonburg, Canada",MMA,29.0,74.0,185.5,"Dec. 11, 2022",73.5,41.0,Aaron Jeffery,NaN


In [97]:
fighter_bio_df.to_csv('data/final/aggregates/All_Fighter_Bios.csv')

#### Get sitemap for fighters

In [98]:
url = 'https://www.sherdog.com/sitemap-fighters.xml'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
r = requests.get(url, headers = header)
r.text

# turn into df
df = pd.DataFrame(r.text.split('\n'))
df.columns = ['url']
df = df[df['url'].str.contains('fighter')]
df

,url
2,<url><loc>https://www.sherdog.com/fighter/Tony...
3,<url><loc>https://www.sherdog.com/fighter/Kurt...
4,<url><loc>https://www.sherdog.com/fighter/Bria...
5,<url><loc>https://www.sherdog.com/fighter/Ken-...
6,<url><loc>https://www.sherdog.com/fighter/Alex...
...,...
19997,<url><loc>https://www.sherdog.com/fighter/Tim-...
19998,<url><loc>https://www.sherdog.com/fighter/Nick...
19999,<url><loc>https://www.sherdog.com/fighter/Curt...
20000,<url><loc>https://www.sherdog.com/fighter/Tomm...


In [99]:
# fighter names, remove  numbers at end of names
df['fighter_first'] = df['url'].str.split('fighter/').str[1].replace('<loc></url>', "").str.split('-').str[0]
df['fighter_last'] = df['url'].str.split('fighter/').str[1].replace('<loc></url>', "").str.split('-').str[-2]

df

,url,fighter_first,fighter_last
2,<url><loc>https://www.sherdog.com/fighter/Tony...,Tony,Galindo
3,<url><loc>https://www.sherdog.com/fighter/Kurt...,Kurt,Rojo
4,<url><loc>https://www.sherdog.com/fighter/Bria...,Brian,Hawkins
5,<url><loc>https://www.sherdog.com/fighter/Ken-...,Ken,Shamrock
6,<url><loc>https://www.sherdog.com/fighter/Alex...,Alexander,Otsuka
...,...,...,...
19997,<url><loc>https://www.sherdog.com/fighter/Tim-...,Tim,Samelstad
19998,<url><loc>https://www.sherdog.com/fighter/Nick...,Nick,Edwards
19999,<url><loc>https://www.sherdog.com/fighter/Curt...,Curtis,Demarce
20000,<url><loc>https://www.sherdog.com/fighter/Tomm...,Tommy,Long


In [100]:
df['sherdog_urls'] = df['url'].str.replace('<loc>', "").str.replace('<url>', '').str.strip()
df

,url,fighter_first,fighter_last,sherdog_urls
2,<url><loc>https://www.sherdog.com/fighter/Tony...,Tony,Galindo,https://www.sherdog.com/fighter/Tony-Galindo-1...
3,<url><loc>https://www.sherdog.com/fighter/Kurt...,Kurt,Rojo,https://www.sherdog.com/fighter/Kurt-Rojo-2</l...
4,<url><loc>https://www.sherdog.com/fighter/Bria...,Brian,Hawkins,https://www.sherdog.com/fighter/Brian-Hawkins-...
5,<url><loc>https://www.sherdog.com/fighter/Ken-...,Ken,Shamrock,https://www.sherdog.com/fighter/Ken-Shamrock-4...
6,<url><loc>https://www.sherdog.com/fighter/Alex...,Alexander,Otsuka,https://www.sherdog.com/fighter/Alexander-Otsu...
...,...,...,...,...
19997,<url><loc>https://www.sherdog.com/fighter/Tim-...,Tim,Samelstad,https://www.sherdog.com/fighter/Tim-Samelstad-...
19998,<url><loc>https://www.sherdog.com/fighter/Nick...,Nick,Edwards,https://www.sherdog.com/fighter/Nick-Edwards-2...
19999,<url><loc>https://www.sherdog.com/fighter/Curt...,Curtis,Demarce,https://www.sherdog.com/fighter/Curtis-Demarce...
20000,<url><loc>https://www.sherdog.com/fighter/Tomm...,Tommy,Long,https://www.sherdog.com/fighter/Tommy-Long-207...


In [101]:
df= df.drop(['url'], axis = 1)
df

,fighter_first,fighter_last,sherdog_urls
2,Tony,Galindo,https://www.sherdog.com/fighter/Tony-Galindo-1...
3,Kurt,Rojo,https://www.sherdog.com/fighter/Kurt-Rojo-2</l...
4,Brian,Hawkins,https://www.sherdog.com/fighter/Brian-Hawkins-...
5,Ken,Shamrock,https://www.sherdog.com/fighter/Ken-Shamrock-4...
6,Alexander,Otsuka,https://www.sherdog.com/fighter/Alexander-Otsu...
...,...,...,...
19997,Tim,Samelstad,https://www.sherdog.com/fighter/Tim-Samelstad-...
19998,Nick,Edwards,https://www.sherdog.com/fighter/Nick-Edwards-2...
19999,Curtis,Demarce,https://www.sherdog.com/fighter/Curtis-Demarce...
20000,Tommy,Long,https://www.sherdog.com/fighter/Tommy-Long-207...


In [102]:
df.to_csv('data/final/aggregates/Sherdog_Fighter_URLs.csv', index = False)

# Scraping UFC.com
### The UFC.COM site has 52 sub-sitemaps. Scraping can also be done this way.

In [103]:
def scrape_ufc_com_from_sitemap(map_num):
    
    #As of now, there are 52 sub-sitemaps

    numz = np.arange(1, map_num, 1)
    numz = numz.astype(str)
    numz = numz.tolist()
    urls = []
    for num in numz:
        url = 'https://www.ufc.com/sitemap.xml?page=' + num
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.find_all('loc'):
            urls.append(link.get_text())

    # keep only links with 'athlete' in them
    urls = [s for s in urls if "/athlete/" in s]
    return urls


In [104]:
all_ufc_com_athletes = scrape_ufc_com_from_sitemap(53)

In [105]:
all_ufc_com_athletes = pd.DataFrame(all_ufc_com_athletes)
all_ufc_com_athletes.columns = ['url']

In [106]:
all_ufc_com_athletes.head()

,url
0,https://www.ufc.com/athlete/shanna-young
1,https://www.ufc.com/athlete/spike-carlyle
2,https://www.ufc.com/athlete/joshua-culibao
3,https://www.ufc.com/athlete/david-dvorak
4,https://www.ufc.com/athlete/darrick-minner


In [107]:
all_ufc_com_athletes['fighter_name'] = all_ufc_com_athletes['url'].str.split('/').str[-1].str.replace('-', ' ').str.title()
all_ufc_com_athletes.head()

,url,fighter_name
0,https://www.ufc.com/athlete/shanna-young,Shanna Young
1,https://www.ufc.com/athlete/spike-carlyle,Spike Carlyle
2,https://www.ufc.com/athlete/joshua-culibao,Joshua Culibao
3,https://www.ufc.com/athlete/david-dvorak,David Dvorak
4,https://www.ufc.com/athlete/darrick-minner,Darrick Minner


In [108]:
all_ufc_com_athletes.to_csv('data/final/aggregates/all_ufcCom_athletes_list.csv', index = False)